<a href="https://colab.research.google.com/github/Gaeseobok/gsb-ml/blob/main/emo7apply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/akmadan/Emotion_Detection_CNN/blob/main/main.py

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from keras.models import load_model
from google.colab import files
from google.colab.patches import cv2_imshow
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image

# face_classifier = cv2.CascadeClassifier(/content/drive/MyDrive/haarcascade_frontalface_default.xml)
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

classifier =load_model('/content/drive/MyDrive/model.h5')

emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

filePath = '/content/drive/MyDrive/mp4/crying2.mp4'


# 영상 객체(파일) 가져오기
cap = cv2.VideoCapture(filePath)

# 프레임을 정수형으로 형 변환
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임

frame_size = (frameWidth, frameHeight)
print('frame_size={}'.format(frame_size))

# 정수 형태로 변환하기 위해 round
round_w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
round_h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) # 카메라에 따라 값이 정상적, 비정상적

# fourcc 값 받아오기, *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
fourcc = cv2.VideoWriter_fourcc('D','I','V','X')

# 웹캠으로 찰영한 영상을 저장하기
# cv2.VideoWriter 객체 생성, 기존에 받아온 속성값 입력
writer = cv2.VideoWriter('emo7_crying2_output.avi', fourcc, fps, (round_w, round_h), isColor = True)

while True:
  try:
    _, frame = cap.read()
    
    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            writer.write(frame) # 영상 데이터만 저장. 소리는 X
            cv2_imshow(frame)

        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

  except:
      print("No face detected")

  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()